In [476]:
import pandas as pd
import html
pd.set_option('display.max_columns', None)

df_player_1617_2425 = pd.read_csv('../../data/processed/pl_16_25_players.csv')
# df_player_1617_2425 = df_player_1617_2425.drop_duplicates()

player_info_1617 = df_player_1617_2425[df_player_1617_2425['season'] == '2016-17']
player_info_1718 = df_player_1617_2425[df_player_1617_2425['season'] == '2017-18']
player_info_1819 = df_player_1617_2425[df_player_1617_2425['season'] == '2018-19']
player_info_1920 = df_player_1617_2425[df_player_1617_2425['season'] == '2019-20']

player_info_2021 = df_player_1617_2425[df_player_1617_2425['season'] == '2020-21']
player_info_2122 = df_player_1617_2425[df_player_1617_2425['season'] == '2021-22']
player_info_2223 = df_player_1617_2425[df_player_1617_2425['season'] == '2022-23']
player_info_2324 = df_player_1617_2425[df_player_1617_2425['season'] == '2023-24']
player_info_2425 = df_player_1617_2425[df_player_1617_2425['season'] == '2024-25']

### Mapping player's position

In [477]:
pos_map = {
    'Goalkeeper': 'GK',
    'Defender': 'DEF',
    'Midfielder': 'MID',
    'Forward': 'FWD',
}

df_list = [player_info_1617, player_info_1718, player_info_1819, player_info_1920, player_info_2021, 
           player_info_2122, player_info_2223, player_info_2324, player_info_2425, df_player_1617_2425]

for df_ in df_list:
    df_['position'] = df_['position'].replace(pos_map)

C:\Users\marse\AppData\Local\Temp\ipykernel_19984\481582971.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['position'] = df_['position'].replace(pos_map)


In [478]:
df_1617 = pd.read_csv('../../data/processed/gameweek/combined_gws_1617.csv')
df_1718 = pd.read_csv('../../data/processed/gameweek/combined_gws_1718.csv')
df_1819 = pd.read_csv('../../data/processed/gameweek/combined_gws_1819.csv')
df_1920 = pd.read_csv('../../data/processed/gameweek/combined_gws_1920.csv')
df_2021 = pd.read_csv('../../data/processed/gameweek/combined_gws_2021.csv')
df_2122 = pd.read_csv('../../data/processed/gameweek/combined_gws_2122.csv')
df_2223 = pd.read_csv('../../data/processed/gameweek/combined_gws_2223.csv')
df_2324 = pd.read_csv('../../data/processed/gameweek/combined_gws_2324.csv')
df_2425 = pd.read_csv('../predict/fpl_price_gw1_.csv')

In [479]:
pos_map = {
    'GKP': 'GK',
}

df_list = [df_1617, df_1718, df_1819, df_1920, df_2021, 
           df_2122, df_2223, df_2324, df_2425]

for df_ in df_list:
    df_['position'] = df_['position'].replace(pos_map)

### Mapping Player's Position & Team (2016/17 - 2019/20) 

In [480]:
# Fungsi untuk membalik nama pemain Korea Selatan
def reverse_korean_name(name):
    parts = name.split(' ')
    if len(parts) == 2:
        return f"{parts[1]} {parts[0]}"
    return name

# Terapkan pada player_info untuk pemain dengan kewarganegaraan 'South Korea'
def apply_reverse_to_korean_players(player_info):
    korean_players = player_info['nationality'] == 'South Korea'
    player_info.loc[korean_players, 'name'] = player_info.loc[korean_players, 'name'].apply(reverse_korean_name)
    return player_info

# Terapkan fungsi ini pada semua season data
player_info_1617 = apply_reverse_to_korean_players(player_info_1617)
player_info_1718 = apply_reverse_to_korean_players(player_info_1718)
player_info_1819 = apply_reverse_to_korean_players(player_info_1819)
player_info_1920 = apply_reverse_to_korean_players(player_info_1920)
player_info_2021 = apply_reverse_to_korean_players(player_info_2021)

player_info_2122 = apply_reverse_to_korean_players(player_info_2122)
player_info_2223 = apply_reverse_to_korean_players(player_info_2223)
player_info_2324 = apply_reverse_to_korean_players(player_info_2324)
player_info_2425 = apply_reverse_to_korean_players(player_info_2425)

In [481]:
from rapidfuzz import process, fuzz

# Fungsi untuk mencocokkan dan mengubah nilai 'name'
def match_and_update_name(df, player_info):
    # Membuat dictionary dari player_info dengan 'name' sebagai kunci
    name_mapping = dict(zip(player_info['name'], player_info['name']))
    
    # Fungsi untuk mencocokkan dan mengubah nama
    def match_name(name, name_mapping):
        match = process.extractOne(name, name_mapping.keys(), scorer=fuzz.ratio, score_cutoff=30)
        if match:
            return match[0]  # Mengembalikan nama yang cocok
        return name  # Mengembalikan nama asli jika tidak ada kecocokan

    # Terapkan fungsi untuk mengubah nilai 'name' pada dataframe
    df['name'] = df['name'].apply(lambda name: match_name(name, name_mapping))

# Terapkan fungsi untuk df_1617, df_1718, df_1819, dan df_1920
match_and_update_name(df_1617, player_info_1617)
match_and_update_name(df_1718, player_info_1718)
match_and_update_name(df_1819, player_info_1819)
match_and_update_name(df_1920, player_info_1920)
match_and_update_name(df_1920, player_info_1920)

match_and_update_name(df_2021, player_info_2021)
match_and_update_name(df_2122, player_info_2122)
match_and_update_name(df_2223, player_info_2223)
match_and_update_name(df_2324, player_info_2324)
match_and_update_name(df_2425, player_info_2425)

In [482]:
# Fungsi untuk mencocokkan dan mengambil nilai 'team' dan 'position'
def match_and_get_info(name, name_to_team, name_to_position):
    match = process.extractOne(name, name_to_team.keys(), scorer=fuzz.ratio, score_cutoff=30)
    if match:
        return name_to_team[match[0]], name_to_position[match[0]]
    return None, None  # Jika tidak ada kecocokan, kembalikan None

# Fungsi untuk menerapkan ke semua dataframe (df_1617, df_1718, df_1819, df_1920)
def apply_team_position(df, player_info):
    # Buat dictionary untuk mapping 'name' ke 'team' dan 'position'
    name_to_team = dict(zip(player_info['name'], player_info['team']))
    name_to_position = dict(zip(player_info['name'], player_info['position']))

    # Terapkan fungsi untuk menambahkan kolom 'team' dan 'position' ke dataframe
    df[['team', 'position']] = df['name'].apply(
        lambda name: match_and_get_info(name, name_to_team, name_to_position)
    ).apply(pd.Series)

# Terapkan ke df_1617, df_1718, df_1819, df_1920
apply_team_position(df_1617, player_info_1617)
apply_team_position(df_1718, player_info_1718)
apply_team_position(df_1819, player_info_1819)
apply_team_position(df_1920, player_info_1920)

apply_team_position(df_2021, player_info_2021)
apply_team_position(df_2122, player_info_2122)
apply_team_position(df_2223, player_info_2223)
apply_team_position(df_2324, player_info_2324)
apply_team_position(df_2425, player_info_2425)

### Adjust Features

In [483]:
new_features_list = ['xP', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded']
df_list = [df_1617, df_1718, df_1819, df_1920]
season_list = ['1617', '1718', '1819', '1920']

for df, season in zip(df_list, season_list):
    
    for feature in new_features_list:
        df[feature] = 0

In [484]:
df_list = [df_1617, df_1718, df_1819, df_1920, df_2021, df_2122, df_2223, df_2324]
season_list = ['16-17', '17-18', '18-19', '19-20', '20-21', '21-22', '22-23', '23-24']

for df, season in zip(df_list, season_list):
    df['season'] = f'20{season}'

In [485]:
df_list = [df_2021, df_2122]
new_features_list = ['expected_assists', 'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded']

for df in df_list:
    for feature in new_features_list:
        df[feature] = 0

In [486]:
cols = ['season', 'name', 'position', 'team', 'assists', 'bonus', 'bps',
        'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
        'goals_scored', 'ict_index', 'influence', 'xP', 'expected_assists', 
        'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded', 
        'kickoff_time', 'minutes', 'opponent_team', 'own_goals', 'penalties_missed', 
        'penalties_saved', 'red_cards', 'saves', 'selected', 'team_a_score',
        'team_h_score', 'threat', 'total_points', 'transfers_balance', 'transfers_in', 
        'transfers_out', 'value', 'was_home', 'yellow_cards', 'GW']

df_1617 = df_1617[cols]
df_1718 = df_1718[cols]
df_1819 = df_1819[cols]
df_1920 = df_1920[cols]
df_2021 = df_2021[cols]
df_2122 = df_2122[cols]
df_2223 = df_2223[cols]
df_2324 = df_2324[cols]
df_2425 = df_2425[cols]

### Merge dataset (2016/17 - 2023/24)

In [487]:
df = pd.concat([df_1617, df_1718, df_1819, df_1920, df_2021, df_2122, df_2223, df_2324, df_2425]).reset_index(drop=True)
df

,season,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016-17,Aaron Cresswell,DEF,West Ham United,0,0,0,0,0.0,454,10,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-08-15T19:00:00Z,0,4,0,0,0,0,0,14023,1.0,2.0,0.0,0,0,0,0,55,0,0,1
1,2016-17,Aaron Lennon,MID,Everton,0,0,6,0,0.3,142,3,0,0,0.9,8.2,0.0,0.0,0.0,0.0,0.0,2016-08-13T14:00:00Z,15,17,0,0,0,0,0,13918,1.0,1.0,0.0,1,0,0,0,60,1,0,1
2,2016-17,Aaron Ramsey,MID,Arsenal,0,0,5,0,4.9,16,8,3,0,3.0,2.2,0.0,0.0,0.0,0.0,0.0,2016-08-14T15:00:00Z,60,9,0,0,0,0,0,163170,4.0,3.0,23.0,2,0,0,0,80,1,0,1
3,2016-17,Abdoulaye Doucouré,MID,Watford,0,0,0,0,0.0,482,7,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-08-13T14:00:00Z,0,13,0,0,0,0,0,1051,1.0,1.0,0.0,0,0,0,0,50,0,0,1
4,2016-17,Abdul Baba,DEF,Chelsea,0,0,0,0,0.0,80,10,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-08-15T19:00:00Z,0,20,0,0,0,0,0,1243,1.0,2.0,0.0,0,0,0,0,55,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197076,2024-25,Chiquinho,FWD,Wolverhampton Wanderers,0,0,0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Arsenal,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1
197077,2024-25,Sasa Kalajdzic,FWD,Wolverhampton Wanderers,0,0,0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Arsenal,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1
197078,2024-25,Fábio Vieira,MID,Arsenal,0,0,0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Arsenal,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1
197079,2024-25,Nathan Fraser,FWD,Wolverhampton Wanderers,0,0,0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Arsenal,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1


### Mapping 'opponent_team' columns

In [488]:
team_id = pd.read_csv('../../data/raw/master_team_list.csv')

id_team_1617 = team_id[team_id['season'] == '2016-17']
id_team_1718 = team_id[team_id['season'] == '2017-18']
id_team_1819 = team_id[team_id['season'] == '2018-19']
id_team_1920 = team_id[team_id['season'] == '2019-20']
id_team_2021 = team_id[team_id['season'] == '2020-21']
id_team_2122 = team_id[team_id['season'] == '2021-22']
id_team_2223 = team_id[team_id['season'] == '2022-23']
id_team_2324 = pd.read_csv('../../data/raw/2023-24/teams.csv')

In [489]:
team_id_1617_map = dict(zip(id_team_1617['team'], id_team_1617['team_name']))
team_id_1718_map = dict(zip(id_team_1718['team'], id_team_1718['team_name']))
team_id_1819_map = dict(zip(id_team_1819['team'], id_team_1819['team_name']))
team_id_1920_map = dict(zip(id_team_1920['team'], id_team_1920['team_name']))
team_id_2021_map = dict(zip(id_team_2021['team'], id_team_2021['team_name']))
team_id_2122_map = dict(zip(id_team_2122['team'], id_team_2122['team_name']))
team_id_2223_map = dict(zip(id_team_2223['team'], id_team_2223['team_name']))
team_id_2324_map = dict(zip(id_team_2324['id'], id_team_2324['name']))

df.loc[df['season'] == '2016-17', 'opponent_team'] = df.loc[df['season'] == '2016-17', 'opponent_team'].replace(team_id_1617_map)
df.loc[df['season'] == '2017-18', 'opponent_team'] = df.loc[df['season'] == '2017-18', 'opponent_team'].replace(team_id_1718_map)
df.loc[df['season'] == '2018-19', 'opponent_team'] = df.loc[df['season'] == '2018-19', 'opponent_team'].replace(team_id_1819_map)
df.loc[df['season'] == '2019-20', 'opponent_team'] = df.loc[df['season'] == '2019-20', 'opponent_team'].replace(team_id_1920_map)
df.loc[df['season'] == '2020-21', 'opponent_team'] = df.loc[df['season'] == '2020-21', 'opponent_team'].replace(team_id_2021_map)
df.loc[df['season'] == '2021-22', 'opponent_team'] = df.loc[df['season'] == '2021-22', 'opponent_team'].replace(team_id_2122_map)
df.loc[df['season'] == '2022-23', 'opponent_team'] = df.loc[df['season'] == '2022-23', 'opponent_team'].replace(team_id_2223_map)
df.loc[df['season'] == '2023-24', 'opponent_team'] = df.loc[df['season'] == '2023-24', 'opponent_team'].replace(team_id_2324_map)

### Editting 'team' and 'opponent_team' value

In [490]:
replace_team_value = {
    'AFC Bournemouth':'Bournemouth', 
    'Arsenal FC':'Arsenal', 
    'Brighton':'Brighton & Hove Albion', 
    'Burnley FC':'Burnley', 
    'Chelsea FC':'Chelsea',
    'Everton FC':'Everton', 
    'Fulham FC':'Fulham',
    'Leeds':'Leeds United', 
    'Leicester':'Leicester City',
    'Liverpool FC':'Liverpool', 
    'Luton':'Luton Town', 
    'Man City':'Manchester City',
    'Man Utd':'Manchester United', 
    'Middlesbrough FC':'Middlesbrough', 
    'Newcastle':'Newcastle United', 
    'Norwich':'Norwich City', 
    "Nott'm Forest":'Nottingham Forest', 
    'Sheffield Utd':'Sheffield United',  
    'Southampton FC':'Southampton', 
    'Spurs':'Tottenham Hotspur',
    'Sunderland AFC':'Sunderland', 
    'Watford FC':'Watford', 
    'West Brom':'West Bromwich Albion',
    'West Ham':'West Ham United',
    'Wolves':'Wolverhampton Wanderers',
    'Huddersfield':'Huddersfield Town',
    'Hull':'Hull City',
    'Stoke':'Stoke City',
    'Swansea':'Swansea City',
    'Cardiff':'Cardiff City',
}

df['team'] = df['team'].replace(replace_team_value)
df['opponent_team'] = df['opponent_team'].replace(replace_team_value)

In [491]:
df

,season,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016-17,Aaron Cresswell,DEF,West Ham United,0,0,0,0,0.0,454,10,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-08-15T19:00:00Z,0,Chelsea,0,0,0,0,0,14023,1.0,2.0,0.0,0,0,0,0,55,0,0,1
1,2016-17,Aaron Lennon,MID,Everton,0,0,6,0,0.3,142,3,0,0,0.9,8.2,0.0,0.0,0.0,0.0,0.0,2016-08-13T14:00:00Z,15,Tottenham Hotspur,0,0,0,0,0,13918,1.0,1.0,0.0,1,0,0,0,60,1,0,1
2,2016-17,Aaron Ramsey,MID,Arsenal,0,0,5,0,4.9,16,8,3,0,3.0,2.2,0.0,0.0,0.0,0.0,0.0,2016-08-14T15:00:00Z,60,Liverpool,0,0,0,0,0,163170,4.0,3.0,23.0,2,0,0,0,80,1,0,1
3,2016-17,Abdoulaye Doucouré,MID,Watford,0,0,0,0,0.0,482,7,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-08-13T14:00:00Z,0,Southampton,0,0,0,0,0,1051,1.0,1.0,0.0,0,0,0,0,50,0,0,1
4,2016-17,Abdul Baba,DEF,Chelsea,0,0,0,0,0.0,80,10,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-08-15T19:00:00Z,0,West Ham United,0,0,0,0,0,1243,1.0,2.0,0.0,0,0,0,0,55,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197076,2024-25,Chiquinho,FWD,Wolverhampton Wanderers,0,0,0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Arsenal,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1
197077,2024-25,Sasa Kalajdzic,FWD,Wolverhampton Wanderers,0,0,0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Arsenal,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1
197078,2024-25,Fábio Vieira,MID,Arsenal,0,0,0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Arsenal,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1
197079,2024-25,Nathan Fraser,FWD,Wolverhampton Wanderers,0,0,0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Arsenal,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1


In [492]:
df[df['name'].str.contains('Ederson')]

,season,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
23819,2017-18,Ederson Moraes,GK,Manchester City,0,0,24,1,0.0,240,2,0,0,1.5,14.8,0.0,0.0,0.0,0.0,0.00,2017-08-12T16:30:00Z,90,Brighton & Hove Albion,0,0,0,0,2,163725,2.0,0.0,0.0,6,0,0,0,55,0,0,1
24345,2017-18,Ederson Moraes,GK,Manchester City,0,0,17,0,0.0,240,100,2,0,2.1,21.4,0.0,0.0,0.0,0.0,0.00,2017-10-28T14:00:00Z,90,West Bromwich Albion,0,0,0,0,2,254244,3.0,2.0,0.0,1,13924,25454,11530,55,0,0,10
24909,2017-18,Ederson Moraes,GK,Manchester City,0,0,12,0,0.0,240,110,1,0,1.6,15.8,0.0,0.0,0.0,0.0,0.00,2017-11-05T14:15:00Z,90,Arsenal,0,0,0,0,2,252018,1.0,3.0,0.0,2,-5643,7643,13286,55,1,0,11
25477,2017-18,Ederson Moraes,GK,Manchester City,0,0,22,1,0.0,240,116,0,0,0.5,5.0,0.0,0.0,0.0,0.0,0.00,2017-11-18T15:00:00Z,90,Leicester City,0,0,0,0,1,258331,2.0,0.0,0.0,6,308,10671,10363,55,0,0,12
26046,2017-18,Ederson Moraes,GK,Manchester City,0,0,9,0,0.0,240,122,1,0,0.4,3.6,0.0,0.0,0.0,0.0,0.00,2017-11-26T16:00:00Z,90,Huddersfield Town,0,0,0,0,0,271408,2.0,1.0,0.0,2,8449,13089,4640,55,0,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193863,2023-24,Ederson Moraes,GK,Manchester City,0,0,23,1,0.0,352,59,0,0,2.3,23.2,4.2,0.0,0.0,0.0,1.03,2023-09-23T14:00:00Z,90,Nottingham Forest,0,0,0,0,3,2084576,0.0,2.0,0.0,6,-5817,82721,88538,56,1,1,6
194619,2023-24,Ederson Moraes,GK,Manchester City,0,0,7,0,0.0,352,70,2,0,0.2,2.4,4.0,0.0,0.0,0.0,0.44,2023-09-30T14:00:00Z,90,Wolverhampton Wanderers,0,0,0,0,0,2077448,1.0,2.0,0.0,1,-22722,46613,69335,56,0,0,7
195391,2023-24,Ederson Moraes,GK,Manchester City,0,0,11,0,0.0,352,71,1,0,1.5,15.4,3.2,0.0,0.0,0.0,0.41,2023-10-08T15:30:00Z,90,Arsenal,0,0,0,0,1,1957237,0.0,1.0,0.0,1,-119732,14604,134336,56,0,1,8
196119,2023-24,Ederson Moraes,GK,Manchester City,0,0,0,0,0.0,352,86,0,0,0.0,0.0,1.7,0.0,0.0,0.0,0.00,2023-10-21T14:00:00Z,0,Brighton & Hove Albion,0,0,0,0,0,1927616,1.0,2.0,0.0,0,-58929,17069,75998,56,1,0,9


### Fixing Value (In Case There's a Wrong Assigned Value)

### Fixing Players Name

Chelsea

In [493]:
df.loc[(df['name'] == 'Alisson Becker') & (df['team'] != 'Liverpool'), 'team'] = 'Liverpool'
df.loc[(df['name'] == 'Trent Alexander-Arnold') & (df['team'] != 'Liverpool'), 'team'] = 'Liverpool'

# df['name'].replace('Neto', 'Pedro Neto')

Everton

In [494]:
df.loc[(df['name'] == 'Eric Bailly') & (df['team'] == 'Everton'), 'team'] = 'Manchester United'

Leeds United

In [495]:
df.loc[(df['name'] == 'Kristoffer Ajer') & (df['team'] != 'Brentford'), 'team'] = 'Brentford'
df.loc[(df['name'] == 'Thiago Alcântara') & (df['team'] == 'Leeds United'), 'team'] = 'Liverpool'
df.loc[(df['name'] == 'Manuel Akanji') & (df['team'] == 'Leeds United'), 'team'] = 'Manchester City'
df.loc[(df['name'] == 'Nathan Aké') & (df['team'] == 'Leeds United'), 'team'] = 'Manchester City'
df.loc[(df['name'] == 'Cameron Archer') & (df['team'] == 'Leeds United'), 'team'] = 'Aston Villa'

Leicester City

In [496]:
df.loc[(df['name'] == 'Pierre-Emerick Aubameyang') & (df['team'] == 'Leicester City'), ['position', 'team']] = ['FWD', 'Arsenal']

Liverpool

In [497]:
df.loc[(df['name'] == 'Allan') & (df['team'] == 'Liverpool'), 'team'] = 'Everton'
df.loc[(df['name'] == 'André Gomes') & (df['team'] == 'Liverpool'), 'team'] = 'Everton'

Man City

In [498]:
df.loc[(df['name'] == 'Pierre-Emerick Aubameyang') & (df['team'] == 'Manchester City'), 'team'] = 'Chelsea'
df.loc[(df['name'] == 'Steven Alzate') & (df['team'] == 'Manchester City'), 'team'] = 'Brighton & Hove Albion'
df.loc[(df['name'] == 'Yasin Ayari') & (df['team'] == 'Manchester City'), 'team'] = 'Brighton & Hove Albion'

West Ham

In [499]:
df.loc[(df['name'] == 'Nathan Aké') & (df['team'] == 'West Ham United'), 'team'] = 'Manchester City'

### Save Cleaned Dataset

In [507]:
df.to_csv('../../data/processed/final-dataset.csv', index=False)